### This notebook is used to test a pipeline to fit the synthbeam peaks from the observation of a calibration source.

In [ ]:
# Allows the modification of the files imported without having to restart kernel or re-import them
%load_ext autoreload
%autoreload 2

In [ ]:
##### %matplotlib notebook
# %matplotlib inline
# to have interactive plots : pip install ipympl
%matplotlib ipympl
from matplotlib import rc
rc('figure',figsize=(16,4))
rc('font',size=12)
rc('text',usetex=False)
rc('image', cmap='viridis')

import qubic.lib.Calibration.Qfiber as ft
import qubic.lib.Fitting.QsynthesizedBeam as sbfit


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy.ndimage.filters as f
import pickle
import os
import fitsio

from scipy.fft import fft2, ifft2, fftfreq
import time

from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float

from qubicpack.qubicfp import qubicfp



In [ ]:
conv_reso_fwhm = 2.35482

In [ ]:
mydatadir3 = "/Users/huchet/Documents/code/data/"

In [ ]:
im = img_as_float(data.coins())

# image_max is the dilation of im with a 20*20 structuring element
# It is used within peak_local_max function
image_max = ndi.maximum_filter(im, size=20, mode='constant')

# Comparison between image_max and im to find the coordinates of local maxima
coordinates = peak_local_max(im, min_distance=20)

# display results
fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(im, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image_max, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('Maximum filter')

ax[2].imshow(im, cmap=plt.cm.gray)
ax[2].autoscale(False)
ax[2].plot(coordinates[:, 1], coordinates[:, 0], 'r.')
ax[2].axis('off')
ax[2].set_title('Peak local max')

fig.tight_layout()

plt.show()

In [ ]:
peak_file = "/Users/huchet/qubic/qubic/calfiles/fitted_peaks.fits"
data, header = fitsio.read(peak_file, header=True)

In [ ]:
fits=fitsio.FITS(peak_file)

In [ ]:
print(fits)

print(fits[0])

In [ ]:
os.uname()[4]

In [ ]:
#Set paths
if os.getcwd() == "/Users/huchet/qubic/qubic/scripts/users/Alexandre/Calibration":
    dirfiles_ = "/Users/huchet/Documents/code/data/ComissioningTD/calib_lab/Synthesized_Beams_Files/"
else:
    raise ValueError("File path unknown.")

In [ ]:
def get_K(Nx, Ny): # already in MoonProject
    '''
    Parameters
    ----------
    hd : map header

    Returns
    -------
    Kx : 2D numpy array (Nx,Ny)
        K values for x dimension for the map.
    Ky : 2D numpy array (Nx,Ny)
        K values for y dimension for the map.
    K : 2D numpy array (Nx,Ny)
        K values for the map.
    '''
    Kx, Ky = np.meshgrid(fftfreq(Nx, d=1/Nx), fftfreq(Ny, d=1/Ny),indexing='ij') # error fixed
    K=np.sqrt(Kx**2 + Ky**2)
    return Kx, Ky, K

def get_Kbin(deltaK, K): # already in MoonProject
    Kmax = np.ceil(np.max(K))
    k = np.arange(3 + deltaK/2, Kmax + deltaK - 1, deltaK)
    Kbin = np.concatenate(([0, 1.5], k[:-2], [Kmax]))  # same def as JB bins (the middle of the bins are JB kp) except for the last bin
    return Kbin

def get_ft_phase(lobe_pos, Nx, Ny): # already in MoonProject
    '''
    Parameters
    ----------
    lobe_pos : int tuple (2)
        x position and y position of the lobe.
    hd : map header

    Returns
    -------
    ft_phase : double
        corrective ft_phase of beam.
    '''
    px = lobe_pos[0]
    py = lobe_pos[1]
    x, y = np.meshgrid(np.arange(Nx), np.arange(Ny), indexing='ij')
    ft_phase = np.exp(-2*np.pi*1j*x*round(-px)/Nx)*np.exp(-2*np.pi*1j*y*round(-py)/Ny)
    return ft_phase


def cos_window(Nx, Ny, lx=None,ly=None): # already in MoonProject
    """
    Window to mutliply with the image to apodise it.
    lx = Nx*0.05/2
    ly = Ny*0.05/2  # donc 2,5% de l'image de chaque côté
    """
    if lx==None:
        lx=Nx*0.05/2
    if ly==None:
        ly=Ny*0.05/2
    result = np.ones((Nx,Ny))
    X,Y = np.meshgrid(np.arange(Nx),np.arange(Ny),indexing='ij')
    whx = X <= lx
    result[whx]=1/2.*(1-np.cos(np.pi/lx*X[whx]))
    whx = X >= Nx-1-lx
    result[whx]=1/2.*(1-np.cos(np.pi/lx*(Nx-1-X[whx])))
    why = Y <= ly
    result[why]=result[why]*1/2.*(1-np.cos(np.pi/ly*Y[why]))  # pour faire les coins aussi
    why = Y >= Ny-1-ly
    result[why]=result[why]*1/2.*(1-np.cos(np.pi/ly*(Ny-1-Y[why])))  # pour faire les coins aussi
    return result

def gauss2D(Nx, Ny, x0, y0, reso, amp=None, normal=True): # reso is the fwhm  # already in MoonProject
    # don't forget to convert all values (x0, y0, reso) in pixel space
    x, y = np.meshgrid(np.arange(Nx), np.arange(Ny), indexing='ij')
    if len(reso) == 1:
        reso = np.array([reso, reso])
    sig = reso / conv_reso_fwhm
    res = np.exp(-(x - x0)**2/(2*sig[0]**2)) * np.exp(-(y - y0)**2/(2*sig[1]**2))
    if normal:
        return(res/np.sum(res))
    else:
        return amp*res
    
def get_filtmapsn(mapj, nKbin, K, Kbin, Kcent, ft_beam_map, ft_phase): # already in MoonProject
    ftmapj = fft2(mapj)
    result = np.zeros((nKbin))
    modu2 = ftmapj*np.conj(ftmapj)
    for i in range(nKbin):
        iKbin = np.logical_and(K>=Kbin[i], K<Kbin[i + 1])
        if len(K[iKbin])>0:
            result[i] = np.abs(np.mean(modu2[iKbin]))
        else:
            print("Kbin [{}, {}] is empty.".format(Kbin[i], Kbin[i + 1]))
    gp = np.interp(K, Kcent, result)   # Pk bins interpolated
    
    ftfilt = np.conj(ft_beam_map)/gp
    normfilt = np.sum(np.abs(ft_beam_map)**2/gp)
    filtmapsn = np.real(ifft2(ftfilt*ftmapj*ft_phase)/np.sqrt(normfilt))  # M convol T / sigma
    return filtmapsn

In [ ]:
def fit_peaks_sb(flatmap, doplot=False):
    reso = np.array([9, 6]) # size pixel in arcmin
    Ni = len(flatmap)
    Nj = len(flatmap[0])
    lobe_pos = (Ni/2, Nj/2)
    _, _, K = get_K(Ni, Nj)
    ft_phase = get_ft_phase(lobe_pos, Ni, Nj)
    border_size_i = Ni*0.1 # Ni = xs
    border_size_j = Nj*0.1 # Nj = xs
    cos_win = cos_window(Ni, Nj, lx=border_size_i, ly=border_size_j)
    deltaK = 1
    Kbin = get_Kbin(deltaK, K)
    nKbin = len(Kbin) - 1  # nb of bins
    Kcent = (Kbin[:-1] + Kbin[1:])/2
    size_pix = reso/60 # degree
    # reso_instr = 0.92 # degree
    reso_img = 1.036 # degree # test
    ft_shape = fft2(gauss2D(Ni, Nj, x0=lobe_pos[0], y0=lobe_pos[1], reso=reso_img/size_pix, normal=True))

    filtmapsn = get_filtmapsn(flatmap * cos_win, nKbin, K, Kbin, Kcent, ft_shape, ft_phase)

    # image_max is the dilation of im with a 20*20 structuring element
    # It is used within peak_local_max function
    size_filt = 25
    image_max = ndi.maximum_filter(filtmapsn, size=size_filt, mode='constant')

    # Comparison between image_max and im to find the coordinates of local maxima
    coordinates = peak_local_max(filtmapsn, min_distance=size_filt, threshold_rel=1e-2, threshold_abs=1e-5, exclude_border=False)

    if doplot:

        plt.figure()
        plt.imshow(np.real(ifft2(ft_shape)), cmap=plt.cm.gray)
        plt.show()

        # display results
        fig, axes = plt.subplots(1, 4, figsize=(8, 3), sharex=True, sharey=True)
        ax = axes[0]
        ax.imshow(flatmap, cmap=plt.cm.gray)
        ax.axis('off')
        ax.set_title('Original')

        ax = axes[1]
        ax.imshow(filtmapsn, cmap=plt.cm.gray)
        ax.axis('off')
        ax.set_title('Gaussian S/N map')

        ax = axes[2]
        ax.imshow(image_max, cmap=plt.cm.gray)
        ax.axis('off')
        ax.set_title('Maximum filter')

        ax = axes[3]
        ax.imshow(flatmap, cmap=plt.cm.gray)
        ax.autoscale(False)
        ax.plot(coordinates[:, 1], coordinates[:, 0], 'r.')
        ax.axis('off')
        ax.set_title('Peak local max')

        fig.tight_layout()
        plt.show()
    return coordinates

In [ ]:
def read_data(datadir, remove_t0=True): # already in MoonProject
    """
    Reads QUBIC raw data: time and TOD, as well azimuth, elevation and 
    their corresponding time

    Parameters
    ----------
    datadir : string
        Full path of the directory where the raw data is stored
        ex/ '/Volumes/QubicData/Calibration/2022-07-14/
    remove_t0 : bool
        subtracts the time of the first sample to the time vector, by default True.

    Returns
    -------
    tt : time for TOD
    tod : the TODs for all detectors
    thk : time for housekeeping data
    az : azimuth of the mount
    el : elevation of the mount
    """
    
    a = qubicfp()
    a.read_qubicstudio_dataset(datadir)
    tt, alltod = a.tod()
    az = a.azimuth()
    el = a.elevation()
    Tbath = a.Tbath
    thk = a.timeaxis(datatype='hk')
    tinit = tt[0]
    if remove_t0:
        ### We remove tt[0]
        tinit = tt[0]
        tt -= tinit
        thk -= tinit
    del(a)
    return tt, alltod, thk, az, el, tinit, Tbath

In [ ]:
dirfiles = dirfiles_ + "130GHz-2019-04-18/"
# dirfiles = dirfiles_ + "140GHz-2019-04-10/"
# dirfiles = dirfiles_ + "150GHz-2019-04-06/"
# dirfiles = dirfiles_ + "160GHz-2019-04-08/"
# dirfiles = dirfiles_ + "170GHz-2019-04-14/"

# Get the data
freq_source = int(dirfiles.split("GHz")[0][-3:]) #130 

# dirfreq = '150GHz-2019-04-06/'
# dirallfreqs = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
# dirfiles = dirallfreqs + dirfreq

c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50

# for TESNum in np.arange(256) + 1:
TESNum = 93
# reload(sbfit)
print(TESNum)
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, azmin = azmin, azmax=azmax)


# plt.figure()
# plt.imshow(flatmap,
#       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], aspect='equal',
#       vmin=-3000, vmax=3000)
# # plt.colorbar()
# plt.show()

coords = fit_peaks_sb(flatmap, doplot=True)

In [ ]:
allTESNum = np.arange(256) + 1
allcoords_pix = []
for TESNum in allTESNum:
    print(TESNum)
    flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, azmin = azmin, azmax=azmax)
    allcoords_pix.append(fit_peaks_sb(flatmap, doplot=False))
    

In [ ]:
#### Offsets from Créidhe
quadrant = 3 # for TD, quadrant = 3; formerly "pointing_offsets_fixed_hole.pickle"
# import pickle
offsets = pickle.load( open( mydatadir3 + 'pointing_offsets_Q{}.pickle'.format(quadrant), 'rb') )
print(np.shape(offsets))

mytesn = np.array(allTESNum)
xycreidhe = offsets[mytesn -1 , :]
print(np.shape(xycreidhe))

######## Here we apply the inversion betwwen Az and El ##############
invert_azel = True # was due to an inversion in the numbering of TES, or was it? It doesn't fit well with the inversion
if invert_azel is True:
    xycreidhe = np.flip(xycreidhe, axis=1)
# print(xycreidhe[0])

invert_az = True # might be an error in the data taking (orientation east/west)?
if invert_az is False:
    xycreidhe[:, 0] = -xycreidhe[:, 0]
# print(xycreidhe[0])

invert_el = False # might be because we do el - elmoon in our data?
if invert_el is True:
    xycreidhe[:, 1] = -xycreidhe[:, 1]

In [ ]:
plt.figure()
for iTES, TESNum in enumerate(allTESNum):
    
    peak_coords = np.zeros(np.shape(allcoords_pix[iTES]))
    for ipeak, peak_coords_pix in enumerate(allcoords_pix[iTES]):
        peak_coords[ipeak] = np.array([az[peak_coords_pix[1]], el[peak_coords_pix[0]]]) # we go from imshow to plot
    theoretical_coordinates = xycreidhe[iTES, None, :]
    plot_coords = peak_coords - theoretical_coordinates
    # plt.plot(allcoords_pix[iTES][:, 1], allcoords_pix[iTES][:, 0], 'b.')
    plt.plot(plot_coords[:, 0], plot_coords[:, 1], 'r.')
    # print(theoretical_coordinates)
    # print(peak_coords)
    # azer
plt.show()


In [ ]:
npix = len(np.ravel(flatmap))
alldata_renorm = np.zeros((256, npix))
for i in range(256):
    flatmap, az, el = sbfit.get_flatmap(i+1, dirfiles, azmin=azmin, azmax=azmax)
    alldata_renorm[i, :] = np.ravel((flatmap - np.mean(flatmap)) / np.std(flatmap))  

In [ ]:
##### Estimating the median of all SB images - to be removed - Origin still unexplained...
med = np.reshape(np.median(alldata_renorm,axis=0),np.shape(flatmap))
mm = np.mean(med)
ss = np.std(med)

plt.figure()
plt.imshow(np.reshape(med, np.shape(flatmap)),
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal')
# plt.colorbar()
plt.title('Median over all S.B. images')
plt.show()

In [ ]:
# reload(sbfit)
TESNum = 93
flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)
# flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles)

mm, ss = ft.meancut(flatmap, 3)
plt.subplot(1,2,1)
plt.imshow(flatmap,
       extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
       aspect='equal',
       vmin=mm-ss, 
       vmax=mm+10*ss)
# plt.colorbar()
plt.title('TES #{}'.format(TESNum))
plt.show()

In [ ]:
# %%script echo skipping

from matplotlib.widgets import Button

# fig, axs = plt.subplots(1, 4, figsize=(10, 3), width_ratios=(1, 1, 1, 0.05))
fig, axs = plt.subplots(1, 2, figsize=(7, 7), width_ratios=(1, 0.05))

# xs = 201
# reso = 6

class Index:

    def __init__(self, TES_num, fig, axes, prev_selec, verbose=False):
        self.fig = fig
        self.prev_selec = prev_selec
        self.isok_arr = np.zeros(len(TES_num), dtype=bool)
        self.ind = -1
        self.axs = axes
        self.verbose = verbose
        self.goNext()

    
    def test_ind(self):
        if self.ind >= len(self.isok_arr):
            if self.verbose:
                print("Not good index")
            return False
        elif self.prev_selec[self.ind] == False:
            if self.verbose:
                print("Already rejected TES")
            self.isok_arr[self.ind] = False
            self.goNext()
        else:
            return True
    
    def goNext(self):
        self.ind += 1
        if self.verbose:
            print(self.ind)
        if not self.test_ind():
            return
        if self.verbose:
            print("TES {}".format(self.ind + 1))
        flatmap, az, el = sbfit.get_flatmap(self.ind + 1, dirfiles, remove=med, azmin=azmin, azmax=azmax)
        # print("ok", flush=True)
        ax = axs[0]
        ax.clear()
        im = ax.imshow(flatmap,
            extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
            aspect='equal',
            # vmin=mm-ss, 
            # vmax=mm+10*ss
        )
        cax = axs[-1]
        cax.clear()
        plt.colorbar(im, cax=cax)
        plt.subplots_adjust(hspace=0)
        self.fig.suptitle("TES {}".format(self.ind + 1))
        plt.draw()

    def good(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = True
        self.goNext()


    def bad(self, event):
        if not self.test_ind():
            return
        self.isok_arr[self.ind] = False
        self.goNext()


allTESNum = np.arange(256) + 1
test_ifok = np.ones_like(allTESNum, dtype=bool)
# callback = Index(allTESNum, fig, axs, visibly_ok_arr, verbose=False)
callback = Index(allTESNum, fig, axs, np.full_like(allTESNum, True), verbose=False)
axbad = fig.add_axes([0.7, 0.05, 0.1, 0.075])
axgood = fig.add_axes([0.81, 0.05, 0.1, 0.075])
bgood = Button(axgood, 'Good')
bgood.on_clicked(callback.good)
bbad = Button(axbad, 'Bad')
bbad.on_clicked(callback.bad)

plt.show()

In [ ]:
%%script echo skipping
# reload(sbfit)
for TESNum in range(1, 257):
    print(TESNum)
    flatmap, az, el = sbfit.get_flatmap(TESNum, dirfiles, remove=med, azmin=azmin, azmax=azmax)
    # plt.figure()
    # plt.subplot(1,2,1)
    # plt.imshow(flatmap,
    #     extent=[np.min(az)*c50, np.max(az)*c50, np.min(el), np.max(el)], 
    #     aspect='equal',
    #     # vmin=mm-ss, 
    # )
    # # plt.colorbar()
    # plt.title('TES #{}'.format(TESNum))
    # plt.show()
    
    #### Instantiate a model
    sbfitmodel3 = sbfit.SbModelIndepPeaks(nrings=2, common_fwhm=True, no_xy_shift=False, distortion=False)
    
    fit, xypeaks = sbfit.fit_sb(flatmap, az, el, sbfitmodel3, verbose=False, resample=False, 
                            doplot=False, extra_title='TES #{}'.format(TESNum))
    thefile = open(dirfiles + '/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fit-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[1], thefile)
    thefile.close
    
    thefile = open(dirfiles + '/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/fiterr-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(fit[2], thefile)
    thefile.close
    
    thefile = open(dirfiles +'/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
#     thefile = open('/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/xypeaks-TES{}.pk'.format(TESNum), 'wb')
    pickle.dump(xypeaks, thefile)
    thefile.close
    # zar


In [ ]:
fit_pos = np.load(dirfiles_ + "Measured_beam_zeroth_peak.npy")

In [ ]:
print(fit_pos.shape)

In [ ]:
print(fit_pos[:, 0])

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
# reload(sbfit)
nperpage = 7
nsiglo = 1.
nsighi = 3.
TESmax = 256
fitted_directory = dirfiles +'/FitSB/'
# fitted_directory = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/FitSB/'

with PdfPages(dirfiles + '/allFitSB_{}.pdf'.format(freq_source)) as pdf:
# with PdfPages('/Users/hamilton/Google Drive/QUBIC/Calib-TD/allFitSB.pdf') as pdf:
    for TESNum in range(1, TESmax+1):
        ipage = (TESNum-1) % nperpage
        print(TESNum, ipage)
        if ipage  == 0:
            rc('figure', figsize=(20,28))
        
        ### Read map
        flatmap, az, el, themap, newxxyy = sbfit.get_flatmap(TESNum, dirfiles, 
                                                             remove=med, azmin=azmin, azmax=azmax,
                                                             fitted_directory = fitted_directory)
        mm, ss = ft.meancut(flatmap-themap,3)
        
        subplot(nperpage, 4, 1+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=np.min(flatmap)/10, 
               vmax=np.max(flatmap)/2)
        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 2+ipage*4)
        imshow(flatmap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        sh = newxxyy.shape
        for i in range(sh[1]):
            ax = plot(newxxyy[0,i], newxxyy[1,i], 'r.')

        colorbar()
        title('Input Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 3+ipage*4)
        imshow(themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Fitted Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        subplot(nperpage, 4, 4+ipage*4)
        imshow(flatmap-themap, extent=[np.min(az)*np.cos(np.radians(50)), 
                               np.max(az)*np.cos(np.radians(50)), 
                               np.min(el), np.max(el)],
               vmin=mm-nsiglo*ss, 
               vmax=mm+nsighi*ss)
        colorbar()
        title('Residuals Map TES# {}'.format(TESNum))
        xlabel('Angle in Az direction [deg.]')
        ylabel('Elevation [deg.]')
        
        if (ipage == (nperpage-1)) or TESNum==TESmax:
            tight_layout()
            pdf.savefig()
            clf()
   
    
    
    
